# 1. Preprocesamiento del conjunto

In [1]:
import csv
import pickle
import time

# Cargamos el fichero en rows
rows = []
with open("/kaggle/input/lorawan-files/lorawan_dataset_antwerp.csv") as csvfile:
    reader = csv.reader(csvfile,)
    next(reader)
    for row in reader:
        rows.append(row)

In [2]:
# Inicializamos las variables
devices_pos = []
devices_rssi_from_gateways = []
times = []

In [3]:
from dateutil.parser import parse
import numpy as np
from copy import deepcopy

for elem in rows:
    aux_rssi = np.asarray(deepcopy(elem[:-5]), dtype=int)
    devices_rssi_from_gateways.append(aux_rssi.tolist())

    get_date_obj_ = parse(deepcopy(elem[-5]))
    times.append(deepcopy(int(get_date_obj_.timestamp()*1000)))


    aux_pos = np.asarray(deepcopy(elem[-2:]), dtype=float)
    devices_pos.append([aux_pos[0],aux_pos[1]])

    
# Ordenamos cronológicamente
devices_pos_sorted = [x for _, x in sorted(zip(times, devices_pos))]
devices_rssi_from_gateways_sorted = [x for _, x in sorted(zip(times, devices_rssi_from_gateways))]
times_sorted = [x for _, x in sorted(zip(times, times))]

In [4]:
# Preparamos la división del conjuto en train (80%) y test (20%)
# de forma cronológica
test_index = [False] * len(devices_pos_sorted)
train_size = int(len(devices_pos_sorted) * 0.8)

for i in range(len(devices_pos_sorted)):
  if i > train_size:
    test_index[i] = True

In [5]:
# Creamos los conjuntois de train y de test
train_X = []
test_X = []
train_y = []
test_y = []

import pandas as pd

for i in range(len(devices_pos_sorted)):
  if test_index[i]:
    test_X.append(devices_rssi_from_gateways_sorted[i])
    test_y.append(devices_pos_sorted[i])
  else:
    train_X.append(devices_rssi_from_gateways_sorted[i])
    train_y.append(devices_pos_sorted[i])

# 2. KNN

In [6]:
# Cargamos la función de Haversine
# Permite hallar la distancia en metros entre dos puntos de la tierra dada su latitud y longitud
import geopy.distance
import math

def haversine(lat1, lat2, lon1, lon2):
    return geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km * 1000

# Definimos una función propia de KNN
# Permite hacer KNN considerando únicamente vecinos concretos y no todos los vecinos
from scipy.spatial import distance
def knn_algorithm(k, train_X, test_X, train_y, test_y, considered_index = [], distancia = 'euclidea'):
    min_distances = []
    min_distances_index = []
    
    # Recorremos cada elemento de test
    for i in range(len(test_X)):
        min_distances.append([])
        min_distances_index.append([])
        # Recorremos cada elemento de train que consideramos
        if considered_index != []:
            index_to_consider = considered_index[i]
        else:
            index_to_consider = range(len(train_X))
        for j in index_to_consider:
            # Seleccionamos la métrica para medir la distancia entre vecinos
            if distancia == 'sorensen':
                current_distance = sorensen_distance(train_X[j], test_X[i])
            elif distancia == 'manhattan':
                current_distance = cityblock(train_X[j], test_X[i])
            else: # distancia euclidea
                current_distance = math.dist(train_X[j], test_X[i])
            if len(min_distances[-1]) < k:
                min_distances[-1].append(current_distance)
                min_distances_index[-1].append(j)
                min_distances_index[-1] = [x for _, x in sorted(zip(min_distances[-1], min_distances_index[-1]))]
                min_distances[-1].sort()

            elif min_distances[-1][-1] > current_distance:
                min_distances[-1][-1] = current_distance
                min_distances_index[-1][-1] = j
                min_distances_index[-1] = [x for _, x in sorted(zip(min_distances[-1], min_distances_index[-1]))]
                min_distances[-1].sort()
                
    predict_y = []

    # Hallamos las predicciones del conjunto de test
    for i in range(len(min_distances_index)):
        punto = [0.0,0.0]
        for j in min_distances_index[i]:
            punto[0] += train_y[j][0]
            punto[1] += train_y[j][1]
        num_elems = len(min_distances_index[i])
        punto[0] =1.0*punto[0]/(1.0*num_elems)
        punto[1] =1.0*punto[1]/(1.0*num_elems)
        predict_y.append(deepcopy(punto))
        
    distances_test_predict = []
    for i in range(len(predict_y)):
        distances_test_predict.append(haversine(test_y[i][0],predict_y[i][0],test_y[i][1],predict_y[i][1]))
        
    # Calculamos el error medio
    mean_error = np.mean(distances_test_predict)
    mean_error
            
    return predict_y, mean_error, min_distances_index, min_distances

In [7]:
def get_multiple_KNN_with_min_distances_and_index(ks, min_distances_index, train_y, test_y):
    predicted_ys = []
    mean_errors = []
    for k in ks:
        predict_y = []
        for i in range(min(len(min_distances_index), k)):
            punto = [0.0,0.0]
            for j in min_distances_index[i]:
                punto[0] += train_y[j][0]
                punto[1] += train_y[j][1]
            num_elems = len(min_distances_index[i])
            punto[0] =1.0*punto[0]/(1.0*num_elems)
            punto[1] =1.0*punto[1]/(1.0*num_elems)
            predict_y.append(deepcopy(punto))

        distances_test_predict = []
        for i in range(len(predict_y)):
            distances_test_predict.append(haversine(test_y[i][0],predict_y[i][0],test_y[i][1],predict_y[i][1]))

        # Calculamos el error medio
        mean_error = np.mean(distances_test_predict)
        mean_error

        predicted_ys.append(predict_y)
        mean_errors.append(mean_error)
        print("Para KNN con k = " + str(k) + " el error medio es " + str(mean_error) + " metros.")
    return predicted_ys, mean_errors

In [ ]:
# Calculamos KNN con K = 15 y guardamos las distancias para poder hallarlo para k < 15
import time
k = 15
predicted_ys = []
mean_errors = []
min_distances_index = []
min_distances = []
start = time.time()
predict_y, mean_error,min_distances_index,min_distances = knn_algorithm(k, train_X, test_X, train_y, test_y)
end = time.time()

print("Tiempo de ejecución del KNN = " + str(end-start))

# Con los valores obtenidos, calculamos el error medio de KNN para K de 1 a 15
ks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
predicted_ys = []
mean_errors = []
predicted_ys, mean_errors = get_multiple_KNN_with_min_distances_and_index(ks, min_distances_index, train_y, test_y)

In [ ]:
# Guardamos en ficheros los valores predichos, los errores medios, las distancias mínimas 
# y los índices del conjuntio de train de las distancias mínimas
import pickle

KNN_15_predicted_y = open('KNN_15_predicted_y.obj', 'wb')
pickle.dump(predicted_ys, KNN_15_predicted_y)
KNN_15_predicted_y.close()

KNN_15_mean_error = open('KNN_15_mean_errors.obj', 'wb')
pickle.dump(mean_errors, KNN_15_mean_error)
KNN_15_mean_error.close()

KNN_15_min_index = open('KNN_15_min_index.obj', 'wb')
pickle.dump(min_distances_index, KNN_15_min_index)
KNN_15_min_index.close()

KNN_15_min_dist = open('KNN_15_min_dist.obj', 'wb')
pickle.dump(min_distances, KNN_15_min_dist)
KNN_15_min_dist.close()

In [8]:
import pickle

KNN_15_mean_error = open('/kaggle/input/lorawan-output/KNN_15_mean_errors.obj', 'rb') 
mean_errors = pickle.load(KNN_15_mean_error)
KNN_15_mean_error.close()

for i in range(len(mean_errors)):
    print("Para KNN con k = " + str(i+1) + " el error medio es " + str(mean_errors[i]) + " metros.")

Para KNN con k = 1 el error medio es 432.02246622317 metros.
Para KNN con k = 2 el error medio es 293.41521187256456 metros.
Para KNN con k = 3 el error medio es 245.46220217655608 metros.
Para KNN con k = 4 el error medio es 288.46604348731864 metros.
Para KNN con k = 5 el error medio es 315.953085641042 metros.
Para KNN con k = 6 el error medio es 311.0387996409501 metros.
Para KNN con k = 7 el error medio es 329.4350314310435 metros.
Para KNN con k = 8 el error medio es 346.7588407609875 metros.
Para KNN con k = 9 el error medio es 325.96238469145277 metros.
Para KNN con k = 10 el error medio es 365.96681832473485 metros.
Para KNN con k = 11 el error medio es 364.45502217886445 metros.
Para KNN con k = 12 el error medio es 349.5102156154953 metros.
Para KNN con k = 13 el error medio es 362.4640369187582 metros.
Para KNN con k = 14 el error medio es 355.1635881188382 metros.
Para KNN con k = 15 el error medio es 350.6831811179458 metros.


## 2.1 Uso de índices para optimizar: Todos coincidentes

In [ ]:
# Calculamos los índices del conjunto de train con rssi != -200
train_indexes_rssi = []

for i in range(len(train_X)):
    train_indexes_rssi.append([])
    for j in range(len(train_X[i])):
        if train_X[i][j] != -200:
            train_indexes_rssi[i].append(j)
            
# Calculamos, para cada elemento de test, los índices del conjunto de train
# que tienen señal en exactamente los mismos índices que el elemento de test
considered_index = []
l = 1
for test_elem in test_X:
    considered_index.append([])
    current_indexes = []
    for i in range(len(test_elem)):
        if test_elem[i] != -200:
            current_indexes.append(i)
    
    for j in range(len(train_X)):
        if len(train_indexes_rssi[j]) == len(current_indexes):
            n = 0
            for i in current_indexes:
                if train_X[j][i] != -200:
                    n+=1
            if n == len(current_indexes):
                considered_index[-1].append(j)

In [ ]:
KNN_test_index_todos_coincidentes = open('KNN_test_index_todos_coincidentes.obj', 'wb')
pickle.dump(considered_index, KNN_test_index_todos_coincidentes)
KNN_test_index_todos_coincidentes.close()

In [ ]:
print(len(considered_index))
n = 0
for elem in considered_index:
    if elem == []:
        n += 1
print("Hay " + str(n) + " elementos del conjunto de test para los que no existe nigún elemento de train con señal en exactamente los mismos índices")

# Creamos las nuevas variables de test para este caso, eliminando aquellso que no consideramos
test_X_todos_coincidentes = [x for x, y in zip(test_X, considered_index) if y != []]
test_y_todos_coincidentes = [x for x, y in zip(test_y, considered_index) if y != []]
considered_index_todos_coincidentes = [x for x in considered_index if x != []]

In [ ]:
# Calculamos KNN con K = 15 y guardamos las distancias para poder hallarlo para k < 15
import time
k = 15
predicted_ys = []
mean_errors = []
min_distances_index = []
min_distances = []
start = time.time()
predict_y, mean_error,min_distances_index,min_distances = knn_algorithm(k, train_X, test_X_todos_coincidentes, train_y, test_y_todos_coincidentes,considered_index_todos_coincidentes)
end = time.time()

print("Tiempo de ejecución del KNN con todos los índices coincidentes = " + str(end-start))

# Con los valores obtenidos, calculamos el error medio de KNN para K de 1 a 15
ks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
predicted_ys = []
mean_errors = []
predicted_ys, mean_errors = get_multiple_KNN_with_min_distances_and_index(ks, min_distances_index, train_y, test_y_todos_coincidentes)

In [ ]:
# Guardamos en ficheros los valores predichos, los errores medios, las distancias mínimas 
# y los índices del conjuntio de train de las distancias mínimas
KNN_15_todos_coincidentes_predicted_y = open('KNN_15_todos_coincidentes_predicted_y.obj', 'wb')
pickle.dump(predicted_ys, KNN_15_todos_coincidentes_predicted_y)
KNN_15_todos_coincidentes_predicted_y.close()

KNN_15_todos_coincidentes_mean_error = open('KNN_15_todos_coincidentes_mean_error.obj', 'wb')
pickle.dump(mean_errors, KNN_15_todos_coincidentes_mean_error)
KNN_15_todos_coincidentes_mean_error.close()

KNN_15_todos_coincidentes_min_index = open('KNN_15_todos_coincidentes_min_index.obj', 'wb')
pickle.dump(min_distances_index, KNN_15_todos_coincidentes_min_index)
KNN_15_todos_coincidentes_min_index.close()

KNN_15_todos_coincidentes_min_dist = open('KNN_15_todos_coincidentes_min_dist.obj', 'wb')
pickle.dump(min_distances, KNN_15_todos_coincidentes_min_dist)
KNN_15_todos_coincidentes_min_dist.close()

In [9]:
import pickle

KNN_15_todos_coincidentes_mean_error = open('/kaggle/input/lorawan-output/KNN_15_todos_coincidentes_mean_error.obj', 'rb') 
mean_errors = pickle.load(KNN_15_todos_coincidentes_mean_error)
KNN_15_todos_coincidentes_mean_error.close()

for i in range(len(mean_errors)):
    print("Para KNN con k = " + str(i+1) + " el error medio es " + str(mean_errors[i]) + " metros.")

Para KNN con k = 1 el error medio es 439.95789747402347 metros.
Para KNN con k = 2 el error medio es 297.38292749799126 metros.
Para KNN con k = 3 el error medio es 248.10734592684057 metros.
Para KNN con k = 4 el error medio es 290.44990130003197 metros.
Para KNN con k = 5 el error medio es 317.5401718912126 metros.
Para KNN con k = 6 el error medio es 312.36137151609233 metros.
Para KNN con k = 7 el error medio es 330.5686644668796 metros.
Para KNN con k = 8 el error medio es 347.7507696673441 metros.
Para KNN con k = 9 el error medio es 326.8440992748809 metros.
Para KNN con k = 10 el error medio es 366.76036144982015 metros.
Para KNN con k = 11 el error medio es 365.1764250198511 metros.
Para KNN con k = 12 el error medio es 350.40849969206596 metros.
Para KNN con k = 13 el error medio es 363.293222220208 metros.
Para KNN con k = 14 el error medio es 355.9335458987559 metros.
Para KNN con k = 15 el error medio es 351.40180837920235 metros.


## 2.2 Uso de índices para optimizar: Al menos 3 coincidentes

In [ ]:
def get_considered_index_at_least_m_equal(m, train_X, test_X):
    considered_index = []
    for test_elem in test_X:
        considered_index.append([])
        current_indexes = []
        for i in range(len(test_elem)):
            if test_elem[i] != -200:
                current_indexes.append(i)

        for j in range(len(train_X)):
            n = 0
            for i in current_indexes:
                if train_X[j][i] != -200:
                    n+=1
                    if n >= m:
                        considered_index[-1].append(j)
                        break
    return considered_index

In [ ]:
# Calculamos, para cada elemento de test, los índices del conjunto de train
# que tienen señal en exactamente los mismos índices que el elemento de test
considered_index = []
considered_index = get_considered_index_at_least_m_equal(3, train_X, test_X)

In [ ]:
KNN_test_index_tres_coincidentes = open('KNN_test_index_tres_coincidentes.obj', 'wb')
pickle.dump(considered_index, KNN_test_index_tres_coincidentes)
KNN_test_index_tres_coincidentes.close()

In [ ]:
print(len(considered_index))
n = 0
for elem in considered_index:
    if elem == []:
        n += 1
print("Hay " + str(n) + " elementos del conjunto de test para los que no existe nigún elemento de train con señal en al menos tres mismos índices")

# Creamos las nuevas variables de test para este caso, eliminando aquellso que no consideramos
test_X_tres_coincidentes = [x for x, y in zip(test_X, considered_index) if y != []]
test_y_tres_coincidentes = [x for x, y in zip(test_y, considered_index) if y != []]
considered_index_tres_coincidentes = [x for x in considered_index if x != []]

In [ ]:
# Calculamos KNN con K = 15 y guardamos las distancias para poder hallarlo para k < 15
k = 15
predicted_ys = []
mean_errors = []
min_distances_index = []
min_distances = []
start = time.time()
predict_y, mean_error,min_distances_index,min_distances = knn_algorithm(k, train_X, test_X_tres_coincidentes, train_y, test_y_tres_coincidentes,considered_index_tres_coincidentes)
end = time.time()

print("Tiempo de ejecución del KNN con tres índices coincidentes = " + str(end-start))

# Con los valores obtenidos, calculamos el error medio de KNN para K de 1 a 15
ks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
predicted_ys = []
mean_errors = []
predicted_ys, mean_errors = get_multiple_KNN_with_min_distances_and_index(ks, min_distances_index, train_y, test_y_tres_coincidentes)

In [ ]:
# Guardamos en ficheros los valores predichos, los errores medios, las distancias mínimas 
# y los índices del conjuntio de train de las distancias mínimas
KNN_15_tres_coincidentes_predicted_y = open('KNN_15_tres_coincidentes_predicted_y.obj', 'wb')
pickle.dump(predicted_ys, KNN_15_tres_coincidentes_predicted_y)
KNN_15_tres_coincidentes_predicted_y.close()

KNN_15_tres_coincidentes_mean_error = open('KNN_15_tres_coincidentes_mean_error.obj', 'wb')
pickle.dump(mean_errors, KNN_15_tres_coincidentes_mean_error)
KNN_15_tres_coincidentes_mean_error.close()

KNN_15_tres_coincidentes_min_index = open('KNN_15_tres_coincidentes_min_index.obj', 'wb')
pickle.dump(min_distances_index, KNN_15_tres_coincidentes_min_index)
KNN_15_tres_coincidentes_min_index.close()

KNN_15_tres_coincidentes_min_dist = open('KNN_15_tres_coincidentes_min_dist.obj', 'wb')
pickle.dump(min_distances, KNN_15_tres_coincidentes_min_dist)
KNN_15_tres_coincidentes_min_dist.close()

In [10]:
import pickle

KNN_15_tres_coincidentes_mean_error = open('/kaggle/input/lorawan-output/KNN_15_tres_coincidentes_mean_error.obj', 'rb') 
mean_errors = pickle.load(KNN_15_tres_coincidentes_mean_error)
KNN_15_tres_coincidentes_mean_error.close()

for i in range(len(mean_errors)):
    print("Para KNN con k = " + str(i+1) + " el error medio es " + str(mean_errors[i]) + " metros.")

Para KNN con k = 1 el error medio es 439.95789747402347 metros.
Para KNN con k = 2 el error medio es 297.38292749799126 metros.
Para KNN con k = 3 el error medio es 248.10734592684057 metros.
Para KNN con k = 4 el error medio es 292.5558230091142 metros.
Para KNN con k = 5 el error medio es 291.33813233538956 metros.
Para KNN con k = 6 el error medio es 316.0838473080919 metros.
Para KNN con k = 7 el error medio es 337.7897985598781 metros.
Para KNN con k = 8 el error medio es 315.51491575679023 metros.
Para KNN con k = 9 el error medio es 319.27293186383133 metros.
Para KNN con k = 10 el error medio es 306.1417707860911 metros.
Para KNN con k = 11 el error medio es 325.3934182198931 metros.
Para KNN con k = 12 el error medio es 319.9654461782252 metros.
Para KNN con k = 13 el error medio es 317.5032951726273 metros.
Para KNN con k = 14 el error medio es 312.55595261937407 metros.
Para KNN con k = 15 el error medio es 298.53296278297415 metros.


## 2.3 Uso de índices para optimizar: Al menos 2 coincidentes

In [ ]:
# Calculamos, para cada elemento de test, los índices del conjunto de train
# que tienen señal en exactamente los mismos índices que el elemento de test
considered_index = []
considered_index = get_considered_index_at_least_m_equal(2, train_X, test_X)

In [ ]:
KNN_test_index_dos_coincidentes = open('KNN_test_index_dos_coincidentes.obj', 'wb')
pickle.dump(considered_index, KNN_test_index_dos_coincidentes)
KNN_test_index_dos_coincidentes.close()

In [ ]:
print(len(considered_index))
n = 0
for elem in considered_index:
    if elem == []:
        n += 1
print("Hay " + str(n) + " elementos del conjunto de test para los que no existe nigún elemento de train con señal en al menos dos mismos índices")

# Creamos las nuevas variables de test para este caso, eliminando aquellso que no consideramos
test_X_dos_coincidentes = [x for x, y in zip(test_X, considered_index) if y != []]
test_y_dos_coincidentes = [x for x, y in zip(test_y, considered_index) if y != []]
considered_index_dos_coincidentes = [x for x in considered_index if x != []]

In [ ]:
# Calculamos KNN con K = 15 y guardamos las distancias para poder hallarlo para k < 15
k = 15
predicted_ys = []
mean_errors = []
min_distances_index = []
min_distances = []
start = time.time()
predict_y, mean_error,min_distances_index,min_distances = knn_algorithm(k, train_X, test_X_dos_coincidentes, train_y, test_y_dos_coincidentes,considered_index_dos_coincidentes)
end = time.time()

print("Tiempo de ejecución del KNN con dos índices coincidentes = " + str(end-start))

# Con los valores obtenidos, calculamos el error medio de KNN para K de 1 a 15
ks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
predicted_ys = []
mean_errors = []
predicted_ys, mean_errors = get_multiple_KNN_with_min_distances_and_index(ks, min_distances_index, train_y, test_y_dos_coincidentes)

In [ ]:
# Guardamos en ficheros los valores predichos, los errores medios, las distancias mínimas 
# y los índices del conjuntio de train de las distancias mínimas
KNN_15_dos_coincidentes_predicted_y = open('KNN_15_dos_coincidentes_predicted_y.obj', 'wb')
pickle.dump(predicted_ys, KNN_15_dos_coincidentes_predicted_y)
KNN_15_dos_coincidentes_predicted_y.close()

KNN_15_dos_coincidentes_mean_error = open('KNN_15_dos_coincidentes_mean_error.obj', 'wb')
pickle.dump(mean_errors, KNN_15_dos_coincidentes_mean_error)
KNN_15_dos_coincidentes_mean_error.close()

KNN_15_dos_coincidentes_min_index = open('KNN_15_dos_coincidentes_min_index.obj', 'wb')
pickle.dump(min_distances_index, KNN_15_dos_coincidentes_min_index)
KNN_15_dos_coincidentes_min_index.close()

KNN_15_dos_coincidentes_min_dist = open('KNN_15_dos_coincidentes_min_dist.obj', 'wb')
pickle.dump(min_distances, KNN_15_dos_coincidentes_min_dist)
KNN_15_dos_coincidentes_min_dist.close()

In [11]:
import pickle

KNN_15_dos_coincidentes_mean_error = open('/kaggle/input/lorawan-output/KNN_15_dos_coincidentes_mean_error.obj', 'rb') 
mean_errors = pickle.load(KNN_15_dos_coincidentes_mean_error)
KNN_15_dos_coincidentes_mean_error.close()

for i in range(len(mean_errors)):
    print("Para KNN con k = " + str(i+1) + " el error medio es " + str(mean_errors[i]) + " metros.")

Para KNN con k = 1 el error medio es 432.02246622317 metros.
Para KNN con k = 2 el error medio es 293.41521187256456 metros.
Para KNN con k = 3 el error medio es 245.46220217655608 metros.
Para KNN con k = 4 el error medio es 288.46604348731864 metros.
Para KNN con k = 5 el error medio es 315.953085641042 metros.
Para KNN con k = 6 el error medio es 311.0387996409501 metros.
Para KNN con k = 7 el error medio es 329.4350314310435 metros.
Para KNN con k = 8 el error medio es 346.7588407609875 metros.
Para KNN con k = 9 el error medio es 325.96238469145277 metros.
Para KNN con k = 10 el error medio es 365.96681832473485 metros.
Para KNN con k = 11 el error medio es 364.45502217886445 metros.
Para KNN con k = 12 el error medio es 349.5102156154953 metros.
Para KNN con k = 13 el error medio es 362.4640369187582 metros.
Para KNN con k = 14 el error medio es 355.1635881188382 metros.
Para KNN con k = 15 el error medio es 350.6831811179458 metros.


## 2.4 Uso de índices para optimizar: Al menos 1 coincidente

In [ ]:
# Calculamos, para cada elemento de la primera mitad de test test, los índices del conjunto de train
# que tienen señal en exactamente los mismos índices que el elemento de test
considered_index = []
considered_index = get_considered_index_at_least_m_equal(1, train_X, test_X[:12000])

In [ ]:
KNN_test_index_un_coincidentes_half_1 = open('KNN_test_index_un_coincidentes_half_1.obj', 'wb')
pickle.dump(considered_index, KNN_test_index_un_coincidentes_half_1)
KNN_test_index_un_coincidentes_half_1.close()

In [ ]:
print(len(considered_index))
n = 0
for elem in considered_index:
    if elem == []:
        n += 1
print("Hay " + str(n) + " elementos del conjunto de test para los que no existe nigún elemento de train con señal en al menos un mismo índice")

# Creamos las nuevas variables de test para este caso, eliminando aquellso que no consideramos
test_X_un_coincidentes = [x for x, y in zip(test_X[:12000], considered_index) if y != []]
test_y_un_coincidentes = [x for x, y in zip(test_y[:12000], considered_index) if y != []]
considered_index_un_coincidentes = [x for x in considered_index if x != []]

In [ ]:
# Calculamos KNN con K = 15 y guardamos las distancias para poder hallarlo para k < 15
k = 15
predicted_ys = []
mean_errors = []
min_distances_index = []
min_distances = []
start = time.time()
predict_y, mean_error,min_distances_index,min_distances = knn_algorithm(k, train_X, test_X_un_coincidentes, train_y, test_y_un_coincidentes,considered_index_un_coincidentes)
end = time.time()

print("Tiempo de ejecución del KNN con un índice coincidente = " + str(end-start))

In [ ]:
# Guardamos en ficheros los valores predichos, los errores medios, las distancias mínimas 
# y los índices del conjuntio de train de las distancias mínimas
KNN_15_un_coincidentes_half_1_min_index = open('KNN_15_un_coincidentes_half_1_min_index.obj', 'wb')
pickle.dump(min_distances_index, KNN_15_un_coincidentes_half_1_min_index)
KNN_15_un_coincidentes_half_1_min_index.close()

KNN_15_un_coincidentes_half_1_min_dist = open('KNN_15_un_coincidentes_half_1_min_dist.obj', 'wb')
pickle.dump(min_distances, KNN_15_un_coincidentes_half_1_min_dist)
KNN_15_un_coincidentes_half_1_min_dist.close()

In [ ]:
# Calculamos, para cada elemento de la primera mitad de test test, los índices del conjunto de train
# que tienen señal en exactamente los mismos índices que el elemento de test
considered_index = []
considered_index = get_considered_index_at_least_m_equal(1, train_X, test_X[12000:])

In [ ]:
KNN_test_index_un_coincidentes_half_2 = open('KNN_test_index_un_coincidentes_half_2.obj', 'wb')
pickle.dump(considered_index, KNN_test_index_un_coincidentes_half_2)
KNN_test_index_un_coincidentes_half_2.close()

In [ ]:
print(len(considered_index))
n = 0
for elem in considered_index:
    if elem == []:
        n += 1
print("Hay " + str(n) + " elementos del conjunto de test para los que no existe nigún elemento de train con señal en al menos un mismo índice")

# Creamos las nuevas variables de test para este caso, eliminando aquellso que no consideramos
test_X_un_coincidentes = [x for x, y in zip(test_X[12000:], considered_index) if y != []]
test_y_un_coincidentes = [x for x, y in zip(test_y[12000:], considered_index) if y != []]
considered_index_un_coincidentes = [x for x in considered_index if x != []]

In [ ]:
# Calculamos KNN con K = 15 y guardamos las distancias para poder hallarlo para k < 15
k = 15
predicted_ys = []
mean_errors = []
min_distances_index = []
min_distances = []
start = time.time()
predict_y, mean_error,min_distances_index,min_distances = knn_algorithm(k, train_X, test_X_un_coincidentes, train_y, test_y_un_coincidentes,considered_index_un_coincidentes)
end = time.time()

print("Tiempo de ejecución del KNN con un índice coincidente = " + str(end-start))


In [ ]:
# Guardamos en ficheros los valores predichos, los errores medios, las distancias mínimas 
# y los índices del conjuntio de train de las distancias mínimas
KNN_15_un_coincidentes_half_2_min_index = open('KNN_15_un_coincidentes_half_2_min_index.obj', 'wb')
pickle.dump(min_distances_index, KNN_15_un_coincidentes_half_2_min_index)
KNN_15_un_coincidentes_half_2_min_index.close()

KNN_15_un_coincidentes_half_2_min_dist = open('KNN_15_un_coincidentes_half_2_min_dist.obj', 'wb')
pickle.dump(min_distances, KNN_15_un_coincidentes_half_2_min_dist)
KNN_15_un_coincidentes_half_2_min_dist.close()

In [12]:
import pickle

KNN_15_un_coincidentes_half_1_min_index = open('/kaggle/input/lorawan-output/KNN_15_un_coincidentes_half_1_min_index.obj', 'rb') 
min_distances_index_1 = pickle.load(KNN_15_un_coincidentes_half_1_min_index)
KNN_15_un_coincidentes_half_2_min_index = open('/kaggle/input/lorawan-output/KNN_15_un_coincidentes_half_2_min_index.obj', 'rb') 
min_distances_index_2 = pickle.load(KNN_15_un_coincidentes_half_2_min_index)

KNN_15_un_coincidentes_half_1_min_dist = open('/kaggle/input/lorawan-output/KNN_15_un_coincidentes_half_1_min_dist.obj', 'rb') 
min_distances_1 = pickle.load(KNN_15_un_coincidentes_half_1_min_dist)
KNN_15_un_coincidentes_half_2_min_dist = open('/kaggle/input/lorawan-output/KNN_15_un_coincidentes_half_2_min_dist.obj', 'rb') 
min_distances_2 = pickle.load(KNN_15_un_coincidentes_half_2_min_dist)

min_distances_index = min_distances_index_1 + min_distances_index_2
min_distances = min_distances_1 + min_distances_2

In [13]:
# Con los valores obtenidos, calculamos el error medio de KNN para K de 1 a 15
ks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
predicted_ys = []
mean_errors = []
predicted_ys, mean_errors = get_multiple_KNN_with_min_distances_and_index(ks, min_distances_index, train_y, test_y)

Para KNN con k = 1 el error medio es 432.02246622317 metros.
Para KNN con k = 2 el error medio es 293.41521187256456 metros.
Para KNN con k = 3 el error medio es 245.46220217655608 metros.
Para KNN con k = 4 el error medio es 288.46604348731864 metros.
Para KNN con k = 5 el error medio es 315.953085641042 metros.
Para KNN con k = 6 el error medio es 311.0387996409501 metros.
Para KNN con k = 7 el error medio es 329.4350314310435 metros.
Para KNN con k = 8 el error medio es 346.7588407609875 metros.
Para KNN con k = 9 el error medio es 325.96238469145277 metros.
Para KNN con k = 10 el error medio es 365.96681832473485 metros.
Para KNN con k = 11 el error medio es 364.45502217886445 metros.
Para KNN con k = 12 el error medio es 349.5102156154953 metros.
Para KNN con k = 13 el error medio es 362.4640369187582 metros.
Para KNN con k = 14 el error medio es 355.1635881188382 metros.
Para KNN con k = 15 el error medio es 350.6831811179458 metros.


In [ ]:
# Guardamos en ficheros los valores predichos, los errores medios, las distancias mínimas 
# y los índices del conjuntio de train de las distancias mínimas
KNN_15_un_coincidentes_predicted_y = open('KNN_15_un_coincidentes_predicted_y.obj', 'wb')
pickle.dump(predicted_ys, KNN_15_un_coincidentes_predicted_y)
KNN_15_un_coincidentes_predicted_y.close()

KNN_15_un_coincidentes_mean_error = open('KNN_15_un_coincidentes_mean_error.obj', 'wb')
pickle.dump(mean_errors, KNN_15_un_coincidentes_mean_error)
KNN_15_un_coincidentes_mean_error.close()

KNN_15_un_coincidentes_min_index = open('KNN_15_un_coincidentes_min_index.obj', 'wb')
pickle.dump(min_distances_index, KNN_15_un_coincidentes_min_index)
KNN_15_un_coincidentes_min_index.close()

KNN_15_un_coincidentes_min_dist = open('KNN_15_un_coincidentes_min_dist.obj', 'wb')
pickle.dump(min_distances, KNN_15_un_coincidentes_min_dist)
KNN_15_un_coincidentes_min_dist.close()

In [ ]:
import pickle

KNN_15_un_coincidentes_mean_error = open('/kaggle/input/lorawan-output/KNN_15_un_coincidentes_mean_error.obj', 'rb') 
mean_errors = pickle.load(KNN_15_un_coincidentes_mean_error)
KNN_15_un_coincidentes_mean_error.close()

for i in range(len(mean_errors)):
    print("Para KNN con k = " + str(i+1) + " el error medio es " + str(mean_errors[i]) + " metros.")

## 2.5 Distancia sorensen con al menos 3 coincidentes

In [15]:
def sorensen_distance(a,b):
    sum_up = 0
    sum_down = 0
    for i in range(len(a)):
        sum_up += abs(a[i]-b[i])
        sum_down += (a[i] + b[i])
    return sum_up / sum_down        

In [16]:
import pickle 
filehandler_KNN_test_index_tres_coincidentes = open('/kaggle/input/lorawan-output/KNN_test_index_tres_coincidentes.obj', 'rb') 
considered_index = pickle.load(filehandler_KNN_test_index_tres_coincidentes)

In [17]:
# Creamos las nuevas variables de test para este caso, eliminando aquellso que no consideramos
test_X_tres_coincidentes = [x for x, y in zip(test_X, considered_index) if y != []]
test_y_tres_coincidentes = [x for x, y in zip(test_y, considered_index) if y != []]
considered_index_tres_coincidentes = [x for x in considered_index if x != []]

In [18]:
# Calculamos KNN con K = 15 y guardamos las distancias para poder hallarlo para k < 15
k = 15
predicted_ys = []
mean_errors = []
min_distances_index = []
min_distances = []
start = time.time()
predict_y, mean_error,min_distances_index,min_distances = knn_algorithm(k, train_X, test_X_tres_coincidentes, train_y, test_y_tres_coincidentes,considered_index_tres_coincidentes, 'sorensen')
end = time.time()

print("Tiempo de ejecución del KNN con tres índices coincidentes = " + str(end-start))

# Con los valores obtenidos, calculamos el error medio de KNN para K de 1 a 15
ks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
predicted_ys = []
mean_errors = []
predicted_ys, mean_errors = get_multiple_KNN_with_min_distances_and_index(ks, min_distances_index, train_y, test_y_tres_coincidentes)

Tiempo de ejecución del KNN con tres índices coincidentes = 415.0178163051605
Para KNN con k = 1 el error medio es 439.95789747437686 metros.
Para KNN con k = 2 el error medio es 402.2537613181579 metros.
Para KNN con k = 3 el error medio es 558.5848935820092 metros.
Para KNN con k = 4 el error medio es 523.7170612750031 metros.
Para KNN con k = 5 el error medio es 733.1132765194125 metros.
Para KNN con k = 6 el error medio es 678.0232771036864 metros.
Para KNN con k = 7 el error medio es 630.649959926534 metros.
Para KNN con k = 8 el error medio es 568.0237971183449 metros.
Para KNN con k = 9 el error medio es 545.1256231623041 metros.
Para KNN con k = 10 el error medio es 509.40919295475186 metros.
Para KNN con k = 11 el error medio es 599.4754365342313 metros.
Para KNN con k = 12 el error medio es 590.986985199362 metros.
Para KNN con k = 13 el error medio es 578.8347327103222 metros.
Para KNN con k = 14 el error medio es 648.3147794687055 metros.
Para KNN con k = 15 el error medio 

## 2.5 Distancia Manhattan con al menos 3 coincidentes

In [19]:
from scipy.spatial.distance import cityblock

In [20]:
import pickle 
filehandler_KNN_test_index_tres_coincidentes = open('/kaggle/input/lorawan-output/KNN_test_index_tres_coincidentes.obj', 'rb') 
considered_index = pickle.load(filehandler_KNN_test_index_tres_coincidentes)

In [21]:
# Creamos las nuevas variables de test para este caso, eliminando aquellso que no consideramos
test_X_tres_coincidentes = [x for x, y in zip(test_X, considered_index) if y != []]
test_y_tres_coincidentes = [x for x, y in zip(test_y, considered_index) if y != []]
considered_index_tres_coincidentes = [x for x in considered_index if x != []]

In [22]:
# Calculamos KNN con K = 15 y guardamos las distancias para poder hallarlo para k < 15
k = 15
predicted_ys = []
mean_errors = []
min_distances_index = []
min_distances = []
start = time.time()
predict_y, mean_error,min_distances_index,min_distances = knn_algorithm(k, train_X, test_X_tres_coincidentes, train_y, test_y_tres_coincidentes,considered_index_tres_coincidentes, 'manhattan')
end = time.time()

print("Tiempo de ejecución del KNN con tres índices coincidentes = " + str(end-start))

# Con los valores obtenidos, calculamos el error medio de KNN para K de 1 a 15
ks = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
predicted_ys = []
mean_errors = []
predicted_ys, mean_errors = get_multiple_KNN_with_min_distances_and_index(ks, min_distances_index, train_y, test_y_tres_coincidentes)

Tiempo de ejecución del KNN con tres índices coincidentes = 329.1016616821289
Para KNN con k = 1 el error medio es 439.95789747402347 metros.
Para KNN con k = 2 el error medio es 284.0567792958599 metros.
Para KNN con k = 3 el error medio es 234.60334226527178 metros.
Para KNN con k = 4 el error medio es 283.19808537484755 metros.
Para KNN con k = 5 el error medio es 283.8519422279762 metros.
Para KNN con k = 6 el error medio es 306.9846496728931 metros.
Para KNN con k = 7 el error medio es 328.769711626514 metros.
Para KNN con k = 8 el error medio es 304.9154460085299 metros.
Para KNN con k = 9 el error medio es 304.2357266892903 metros.
Para KNN con k = 10 el error medio es 292.60828612911024 metros.
Para KNN con k = 11 el error medio es 313.0902503499105 metros.
Para KNN con k = 12 el error medio es 315.21753696228933 metros.
Para KNN con k = 13 el error medio es 308.4692069143178 metros.
Para KNN con k = 14 el error medio es 304.16715637951523 metros.
Para KNN con k = 15 el error m

# 3. Red neuronal

In [23]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Definir una función para crear y entrenar el modelo
def train_model(learning_rate, num_neuronas, dropout, train_X, test_X, train_y, test_y, loss_function = 'mse'):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(num_neuronas, activation='relu', input_shape=(68,)))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(num_neuronas, activation='relu'))
    model.add(tf.keras.layers.Dense(num_neuronas, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(num_neuronas, activation='relu'))
    model.add(tf.keras.layers.Dense(2))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss_function)

    model.fit(train_X, train_y, epochs=500, validation_data=(test_X, test_y), verbose=False)

    return model

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [1]:
learning_rates = [0.001, 0.01, 0.1]
num_neuronas_list = [32, 128]
dropout_valores = [0.2, 0.4, 0.6]

best_model = None
best_error = None

for learning_rate in learning_rates:
    for num_neuronas in num_neuronas_list:
        for dropout_valor in dropout_valores:
            print(f"Entrenando modelo con learning_rate={learning_rate}, num_neuronas={num_neuronas}, dropout_valor={dropout_valor}")
            model = train_model(learning_rate, num_neuronas, dropout_valor, train_X, test_X, train_y, test_y)
            predict_y = model.predict(test_X)
            
            distances_test_predict = []
            for i in range(len(predict_y)):
                distances_test_predict.append(haversine(test_y[i][0],predict_y[i][0],test_y[i][1],predict_y[i][1]))

            # Calculamos el error medio
            print("El error es de " + str(np.mean(distances_test_predict)) + " metros")
            
            model.save("model_lr"+str(learning_rate)+"_nn"+ str(num_neuronas)+"_dr"+str(dropout_valor)+".h5")

            if best_error is None or np.mean(distances_test_predict) < best_error:
                best_error = np.mean(distances_test_predict)
                best_model = model


Entrenando modelo con learning_rate=0.001, num_neuronas=32, dropout_valor=0.2
773/773 [==============================] - 1s 1ms/step
El error es de 1971.6104452864638 metros
Entrenando modelo con learning_rate=0.001, num_neuronas=32, dropout_valor=0.4
773/773 [==============================] - 1s 1ms/step
El error es de 1683.8604890952467 metros
Entrenando modelo con learning_rate=0.001, num_neuronas=32, dropout_valor=0.6
773/773 [==============================] - 1s 1ms/step
El error es de 1907.4081210573868 metros
Entrenando modelo con learning_rate=0.001, num_neuronas=128, dropout_valor=0.2
773/773 [==============================] - 1s 2ms/step
El error es de 1384591.5104975754 metros
Entrenando modelo con learning_rate=0.001, num_neuronas=128, dropout_valor=0.4
773/773 [==============================] - 1s 1ms/step
El error es de 12395.623357161723 metros
Entrenando modelo con learning_rate=0.001, num_neuronas=128, dropout_valor=0.6
773/773 [==============================] - 1s 1m

## 3.1 Función de pérdida: función Haversine

In [28]:
from keras import backend as K
import tensorflow as tf

def degrees_to_radians(deg):
    return deg * 0.017453292519943295

def loss_haversine(observation, prediction):    
    obv_rad = K.map_fn(degrees_to_radians, observation)
    prev_rad = K.map_fn(degrees_to_radians, prediction)

    dlon_dlat = obv_rad - prev_rad 
    v = dlon_dlat / 2
    v = tf.sin(v)
    v = v**2

    a = v[:,0] + K.cos(obv_rad[:,0]) * K.cos(prev_rad[:,0]) * v[:,1] 

    c = K.sqrt(a)
    c = 2* tf.asin(c)
    c = c*6378.1 # radio de la tierra en metros

    final = tf.reduce_sum(c)

    return final*1000

In [29]:
# Entrenamos el mejor modelo con la función de pérdida loss_haversine

model = train_model(0.001, 32, 0.4, train_X, test_X, train_y, test_y, loss_haversine)
predict_y = model.predict(test_X)

distances_test_predict = []
for i in range(len(predict_y)):
    distances_test_predict.append(haversine(test_y[i][0],predict_y[i][0],test_y[i][1],predict_y[i][1]))

# Calculamos el error medio
print("El error es de " + str(np.mean(distances_test_predict)) + " metros")

model.save("model_haversine.h5")


773/773 [==============================] - 1s 1ms/step
El error es de 2643059.4465170344 metros


# 4. Random forest

In [31]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

def random_forest(n_estimators, max_depth, min_samples_split, train_X, train_y):
    # Crear y entrenar el modelo de Random Forest
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)
    model.fit(train_X, train_y)
    
    return model

In [36]:
# Probar diferentes combinaciones de hiperparámetros
n_estimators = [50, 100, 200]
max_depth = [None, 5, 10]
min_samples_split = [2, 5, 10]

best_model = None
best_error = None

for n_estimator in n_estimators:
    for max_depth_actual in max_depth:
        for min_samples_split_actual in min_samples_split:
            print(f"Entrenando modelo con n_estimators={n_estimator}, max_depth={max_depth_actual}, min_samples_split={min_samples_split_actual}")
            model = random_forest(n_estimator, max_depth_actual, min_samples_split_actual, train_X, train_y)
            predict_y = model.predict(test_X)
            
            distances_test_predict = []
            for i in range(len(predict_y)):
                distances_test_predict.append(haversine(test_y[i][0],predict_y[i][0],test_y[i][1],predict_y[i][1]))

            # Calculamos el error medio
            print("El error es de " + str(np.mean(distances_test_predict)) + " metros")

            if best_error is None or np.mean(distances_test_predict) < best_error:
                best_error = np.mean(distances_test_predict)
                best_model = model

Entrenando modelo con n_estimators=50, max_depth=None, min_samples_split=2
El error es de 489.6140779668307 metros
Entrenando modelo con n_estimators=50, max_depth=None, min_samples_split=5
El error es de 479.0538700815534 metros
Entrenando modelo con n_estimators=50, max_depth=None, min_samples_split=10
El error es de 471.7883495723622 metros
Entrenando modelo con n_estimators=50, max_depth=5, min_samples_split=2
El error es de 965.7151393203206 metros
Entrenando modelo con n_estimators=50, max_depth=5, min_samples_split=5
El error es de 969.0437630881667 metros
Entrenando modelo con n_estimators=50, max_depth=5, min_samples_split=10
El error es de 965.5975559592532 metros
Entrenando modelo con n_estimators=50, max_depth=10, min_samples_split=2
El error es de 648.7532987656006 metros
Entrenando modelo con n_estimators=50, max_depth=10, min_samples_split=5
El error es de 650.3392546416533 metros
Entrenando modelo con n_estimators=50, max_depth=10, min_samples_split=10
El error es de 65